# Python Datastream API

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import pandas as pd
import numpy as np
import json

from pyflink.common import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors.kafka import FlinkKafkaProducer, FlinkKafkaConsumer
from pyflink.datastream.formats.json import JsonRowSerializationSchema, JsonRowDeserializationSchema

In [6]:
!curl -O https://repo1.maven.org/maven2/org/apache/flink/flink-connector-kafka/3.1.0-1.18/flink-connector-kafka-3.1.0-1.18.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  465k  100  465k    0     0  4129k      0 --:--:-- --:--:-- --:--:-- 4155k


In [14]:
!curl -O https://repo1.maven.org/maven2/org/apache/flink/flink-sql-connector-kafka/3.1.0-1.18/flink-sql-connector-kafka-3.1.0-1.18.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5465k  100 5465k    0     0  30.8M      0 --:--:-- --:--:-- --:--:-- 31.0M


In [17]:
env = StreamExecutionEnvironment.get_execution_environment()
env.add_jars("file:///Users/chriswong/Projects/kafka-summit-2024/notebook/flink-sql-connector-kafka-3.1.0-1.18.jar")


In [5]:
with open('../config.json') as fp:
    config_obj = json.load(fp)
config_obj

{'flink.servers': '',
 'kafka.brokers': 'localhost:49523',
 'kafka.topic.price.name': 'local.price',
 'kafka.topic.holding.name': 'local.holding',
 'kafka.topic.order.name': 'local.order'}

In [18]:
kafka_brokers = config_obj['kafka.brokers']
price_topic = config_obj['kafka.topic.price.name']

deserialization_schema = JsonRowDeserializationSchema.Builder() \
    .type_info(Types.ROW([Types.STRING(), Types.STRING()])) \
    .build()

kafka_consumer = FlinkKafkaConsumer(
    topics=price_topic,
    deserialization_schema=deserialization_schema,
    properties={'bootstrap.servers': kafka_brokers, 'group.id': 'test_group_1'}
)

kafka_consumer.set_start_from_earliest()

env.add_source(kafka_consumer).print()


Py4JJavaError: An error occurred while calling None.org.apache.flink.streaming.connectors.kafka.FlinkKafkaConsumer.
: java.lang.NoClassDefFoundError: org/apache/kafka/common/serialization/ByteArrayDeserializer
	at org.apache.flink.streaming.connectors.kafka.FlinkKafkaConsumer.setDeserializer(FlinkKafkaConsumer.java:323)
	at org.apache.flink.streaming.connectors.kafka.FlinkKafkaConsumer.<init>(FlinkKafkaConsumer.java:224)
	at org.apache.flink.streaming.connectors.kafka.FlinkKafkaConsumer.<init>(FlinkKafkaConsumer.java:155)
	at org.apache.flink.streaming.connectors.kafka.FlinkKafkaConsumer.<init>(FlinkKafkaConsumer.java:140)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Cons

In [ ]:
env.execute()